In [7]:
import pandas as pd
import panel as pn
import numpy as np
import hvplot.pandas
import seaborn as sns
#basic+bonus部分
#首先打开文件world_wide_COVID_data_daily_data.csv
COVID_data='C:/Users/cx/Desktop/world_wide_COVID_data_daily_data.csv'
COVID_file1=pd.read_csv(COVID_data)
COVID_file=COVID_file1.iloc[::-1]

#将文件中所有国家全部整理成一个list，命名为list_country
list_country1=COVID_file['countriesAndTerritories'].tolist()
list_country=[]
for i in list_country1:
    if not i in list_country:
        list_country.append(i)
        
#将文件中的所有的大洲整理成一个list，命名为list_continent       
list_continent1=COVID_file['continentExp'].tolist()        
list_continent=[]  
for i in list_continent1:
    if not i in list_continent:
        list_continent.append(i)

In [8]:
#利用月份设置IntRangeSlider,y轴的数值设计成 RadioButtonGroup样式，country及continent设计成MultiSelect   
months= pn.widgets.IntRangeSlider(name='months',value=(3,4),start=1, end=13,step=1,width=600)
yvalue=pn.widgets.RadioButtonGroup(name='y',options=['cases','deaths'],width=400)
continent_select=pn.widgets.MultiSelect(name='continents',value=['Asia'],options=list_continent,width=400)
country_select=pn.widgets.MultiSelect(name='countries',value=['China'],options=list_country,width=400)

In [9]:
@pn.depends(continent_select,watch=True)
def updata_countries(continent):
    #建立一个空的dataframe
    df_dataframe=pd.DataFrame(columns=['dateRep','day','month','year','cases','deaths','countriesAndTerritories','geoId','countryterritoryCode','popData2019','continentExp','Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'])
    new_options=[]
    for i in continent:
        data1=COVID_file[COVID_file['continentExp']==i]
        #通过continent_select的value从COVID_file中筛选所需要的数值，逐步加到空的dataframe---df_dataframe中
        df_dataframe=df_dataframe.append(data1)
        list1=df_dataframe['countriesAndTerritories'].tolist()
        #然后将新建的df_dataframe的'countriesAndTerritories'整理成一个list
        #使这个list即new_options成为country_select的option
        #这样就可以通过选定continent_select的value来使country_select中显示相应的值
        for a in list1:
            if not a in new_options:
                new_options.append(a)
    country_select.options=new_options

In [10]:
#设置通过months,yvalue,country_select的值实现plot   
@pn.depends(months,yvalue,country_select,watch=True)
def make_plot(xaxis,yaxis,country):
    #同样先设立一个空的dataframe即df_dataframe
    df_dataframe=pd.DataFrame(columns=['dateRep','day','month','year','cases','deaths','countriesAndTerritories','geoId','countryterritoryCode','popData2019','continentExp','Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'])
    #根据months及country_select的value筛选出相应的数值，逐步加到df_dataframe中
    for b in range(xaxis[0],xaxis[1]):
        file1=COVID_file[COVID_file['month']==b]
        for e in country:
            file2=file1[file1['countriesAndTerritories']==e]
            file2.reset_index(drop=True)
            df_dataframe=df_dataframe.append(file2)
    #将最后得到的df_dataframe绘图
    plot=df_dataframe.hvplot(x='dateRep',rot=90,y=yaxis,kind='line',by='countriesAndTerritories')
    return plot

In [11]:
#根画图同样的方式，建立table
@pn.depends(months,yvalue,country_select,watch=True)
def Data_table(xaxis,yaxis,country):
    df_dataframe=pd.DataFrame(columns=['dateRep','day','month','year','cases','deaths','countriesAndTerritories','geoId','countryterritoryCode','popData2019','continentExp','Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'])
    for b in range(xaxis[0],xaxis[1]):
        file1=COVID_file[COVID_file['month']==b]
        for e in country:
            file2=file1[file1['countriesAndTerritories']==e]
            file2.reset_index(drop=True)
            df_dataframe=df_dataframe.append(file2)
            file2.reset_index(drop=True)
    Data_table=df_dataframe.hvplot.table()
    return Data_table

In [12]:
pn.Column(pn.WidgetBox(months,yvalue,continent_select,country_select,width=800),make_plot,Data_table)

Traceback (most recent call last):
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\pyviz_comms\__init__.py", line 325, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\panel\viewable.py", line 273, in _on_msg
 doc.unhold()
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\bokeh\document\document.py", line 799, in unhold
 self.callbacks.unhold()
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\bokeh\document\callbacks.py", line 396, in unhold
 self.trigger_on_change(event)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\bokeh\document\callbacks.py", line 373, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\bokeh\util\callback_manager.py", line 191, in invoke
 callback(attr, old, new)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\panel\reactive.py", line 301, in _comm_change
 self._process_events({attr: new})
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\panel\reactive.py", line 262, in _process_events
 self.param.set_param(**self_events)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 1904, in set_param
 return self_.update(kwargs)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 1877, in update
 self_._batch_call_watchers()
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 2038, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 2000, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 422, in cb
 return func(*args, **dep_kwargs)
 File "C:\Users\cx\AppData\Local\Temp/ipykernel_11700/976022993.py", line 17, in updata_countries
 country_select.options=new_options
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 351, in _f
 instance_param.__set__(obj, val)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 353, in _f
 return f(self, obj, val)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 1228, in __set__
 obj.param._call_watcher(watcher, event)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 2018, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 2000, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\panel\reactive.py", line 242, in _param_change
 msg = self._process_param_change({event.name: event.new})
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\panel\widgets\select.py", line 167, in _process_param_change
 self.value = [v for v in self.value if isIn(v, values)]
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 351, in _f
 instance_param.__set__(obj, val)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 353, in _f
 return f(self, obj, val)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 1228, in __set__
 obj.param._call_watcher(watcher, event)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 2018, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\param\parameterized.py", line 2000, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\cx\anaconda3\envs\panel\lib\site-packages\panel\param.py", line 751, in _replace_pane
 self._update_inner(new_object)
 File "C:\Users\cx

Column
    [0] WidgetBox(width=800)
        [0] IntRangeSlider(end=13, name='months', start=1, value=(3, 4), value_end=4, value_start=3, width=600)
        [1] RadioButtonGroup(name='y', options=['cases', 'deaths'], value='cases', width=400)
        [2] MultiSelect(name='continents', options=['Africa', 'Asia', ...], value=['Asia'], width=400)
        [3] MultiSelect(name='countries', options=['Zimbabwe', 'Zambia', ...], value=['China'], width=400)
    [1] ParamFunction(function)
    [2] ParamFunction(function)